🔧 1. Install & Import Libraries

In [13]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.optim as optim

from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import timm
import os

# Device setup
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("✅ Using device:", device)
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version (compiled):", torch.version.cuda)
print("GPU count:", torch.cuda.device_count())


✅ Using device: cpu
Torch version: 2.9.1+cpu
CUDA available: False
CUDA version (compiled): None
GPU count: 0


🧠 2. Load DINOv2 & Smart Unfreezing

In [ ]:
# ✅ Load pretrained DINOv2 ViT-Base model
backbone = timm.create_model('vit_base_patch14_dinov2.lvd142m', pretrained=True, num_classes=0).to(device)

# Freeze entire model initially
for param in backbone.parameters():
    param.requires_grad = False

# ✅ Smart unfreezing: last transformer block and final norm layer
if hasattr(backbone, 'blocks'):
    for param in backbone.blocks[-1].parameters():
        param.requires_grad = True

if hasattr(backbone, 'norm'):
    for param in backbone.norm.parameters():
        param.requires_grad = True

# ✅ Print expected input size
print("✅ Input size expected by DINOv2:", backbone.default_cfg['input_size'])


🏗️ 3. Define Classifier Head

In [ ]:
class DinoClassifier(nn.Module):
    def __init__(self, backbone, num_classes):
        super().__init__()
        self.backbone = backbone
        self.head = nn.Sequential(
            nn.Linear(768, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        feats = self.backbone(x)
        return self.head(feats)


🧼 4. Transforms & Dataloaders

In [ ]:
# ✅ Local dataset root
data_path = r"D:\Dermora\dataset"

# ✅ Training data augmentation
train_transform = T.Compose([
    T.RandomResizedCrop(518, scale=(0.8, 1.0)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.ColorJitter(0.2, 0.2, 0.2),
    T.GaussianBlur(kernel_size=3),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406],
                [0.229, 0.224, 0.225])
])

# ✅ Validation/test transforms (no augmentation)
test_transform = T.Compose([
    T.Resize((518, 518)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406],
                [0.229, 0.224, 0.225])
])

# ✅ Load datasets using correct Windows path
train_dataset = ImageFolder(os.path.join(data_path, 'train'), transform=train_transform)
val_dataset   = ImageFolder(os.path.join(data_path, 'val'), transform=test_transform)
test_dataset  = ImageFolder(os.path.join(data_path, 'test'), transform=test_transform)

# ✅ Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,
                          num_workers=2, pin_memory=True, persistent_workers=True, prefetch_factor=2)
val_loader   = DataLoader(val_dataset, batch_size=16, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=16, num_workers=2, pin_memory=True)

# ✅ Class names
class_names = train_dataset.classes
print("✅ Classes:", class_names)


⚙️ 5. Loss, Optimizer, Scheduler

In [ ]:
# Assuming backbone is already defined in Block 2
model = DinoClassifier(backbone, num_classes=3).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.5)


🔁 6. Training, Validation, Evaluation Functions

In [ ]:
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def train(model, loader, epoch):
    model.train()
    total, correct, running_loss = 0, 0, 0

    pbar = tqdm(loader, desc=f"🔁 Training (Epoch {epoch})")
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        running_loss += loss.item()

        pbar.set_postfix(loss=loss.item(), acc=correct / total)

    return correct / total, running_loss / len(loader)


def validate(model, loader, epoch):
    model.eval()
    total, correct = 0, 0

    pbar = tqdm(loader, desc=f"🧪 Validating (Epoch {epoch})")
    with torch.no_grad():
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            pbar.set_postfix(acc=correct / total)

    return correct / total


def evaluate(model, loader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in tqdm(loader, desc="🔍 Evaluating"):
            images = images.to(device)
            outputs = model(images)
            preds = outputs.argmax(dim=1).cpu()
            all_preds.extend(preds.numpy())
            all_labels.extend(labels.numpy())

    print("\n📊 Classification Report:\n")
    print(classification_report(all_labels, all_preds, target_names=class_names))

    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.tight_layout()
    plt.show()


🧠 7. Training Loop with Full Checkpoint

In [ ]:
best_acc = 0
num_epochs = 20

for epoch in range(1, num_epochs + 1):
    print(f"\n📅 Epoch {epoch}/{num_epochs}")

    train_acc, train_loss = train(model, train_loader, epoch)
    val_acc = validate(model, val_loader, epoch)
    scheduler.step(val_acc)

    print(f"✅ Train Acc: {train_acc:.4f} | Loss: {train_loss:.4f}")
    print(f"🧪 Val   Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_acc': best_acc
        }, 'best_dino_checkpoint.pth')
        print("💾 Model checkpoint saved!")


💾 8. Load Best Model & Evaluate

In [ ]:
# 💾 8. Load Best Model & Evaluate

# Load checkpoint from file
checkpoint_path = 'best_dino_checkpoint.pth'
checkpoint = torch.load(checkpoint_path, map_location=device)

# Load weights into model and optimizer
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
best_epoch = checkpoint['epoch']
best_val_acc = checkpoint['best_acc']

print(f"✅ Loaded checkpoint from epoch {best_epoch} with best val accuracy: {best_val_acc:.4f}")

# Final test evaluation
evaluate(model, test_loader)


🧪 📊 Confusion Matrix for Classification Evaluation

In [ ]:
# Load checkpoint (only if file exists!)
# import os

# if os.path.exists('best_dino_checkpoint.pth'):
#     checkpoint = torch.load('best_dino_checkpoint.pth')
#     model.load_state_dict(checkpoint['model_state_dict'])
#     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#     print(f"✅ Loaded checkpoint from epoch {checkpoint['epoch']} with best acc: {checkpoint['best_acc']:.4f}")
# else:
#     print("❌ No checkpoint file found. Train the model first.")
